In [3]:
import requests
import pandas as pd
import numpy as np
import pygsheets
import datetime
from datetime import date
import json
import webbrowser
from time import sleep
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Importing functions
import sys
sys.path.append(r'C:\\Users\\Manuel Elizaldi\\Desktop\\Learning-Testing\\PyStrava\\Scripts')
from Functions import *

In [5]:
# Specify the path to your JSON file
file_path = r'C:\Users\Manuel Elizaldi\Desktop\Learning-Testing\PyStrava\Outputs\all_workouts_json.json'

# Open the JSON file and load its contents into a Python object
with open(file_path, 'r') as json_file:
    all_workouts_json = json.load(json_file)

[{'resource_state': 3, 'athlete': {'id': 58832723, 'resource_state': 1}, 'name': 'Morning Run', 'distance': 3014.0, 'moving_time': 1303, 'elapsed_time': 1373, 'total_elevation_gain': 32.0, 'type': 'Run', 'sport_type': 'Run', 'workout_type': 'None', 'id': 10343078755, 'start_date': '2023-12-07T13:56:12Z', 'start_date_local': '2023-12-07T07:56:12Z', 'timezone': '(GMT-06:00) America/Chicago', 'utc_offset': -21600.0, 'location_city': 'None', 'location_state': 'None', 'location_country': 'United States', 'achievement_count': 1, 'kudos_count': 0, 'comment_count': 0, 'athlete_count': 1, 'photo_count': 0, 'map': {'id': 'a10343078755', 'polyline': 'uqpxDv|esQJZAJOZw@^i@`@_@JGJCXOX[TQTGX?dAV|@x@zDLZbAtAZl@n@HNJxAvBJVHb@B`@APK`@[j@s@v@aBfAg@d@INUPc@H]LSJeA`AOF_@^cAt@IP]d@[Rs@~@GLGb@A\\JTJJFDf@JNJPZJl@@t@Hj@VdA]rAmAvCOTYR[Fa@Ra@`@SJD[P[ZSn@E\\IlAaDT}@B[?Y[yAEa@Ai@G]KWk@?a@a@G_@Ak@@QLg@zCgD`AcAVS`@ORMZYnAe@`@SlAiAbAu@TWX[LWBQC[@SEWOe@AGs@kAY_@WC]?KEIMUg@OOm@mAm@gBa@iBS]GUCS?UDg@FQFYLOj@g@J{@', 'res

In [6]:
pd.json_normalize(all_workouts_json, 'segment_efforts')

,id,resource_state,name,elapsed_time,moving_time,start_date,start_date_local,distance,start_index,end_index,...,segment.start_latlng,segment.end_latlng,segment.elevation_profile,segment.climb_category,segment.city,segment.state,segment.country,segment.private,segment.hazardous,segment.starred
0,3167848617670903142,2,WC Parking to 1st Bridge,116,94,2023-12-07T13:56:50Z,2023-12-07T07:56:50Z,253.8,15,39,...,"[30.401144, -97.684408]","[30.401298, -97.686722]",None,0,Austin,Texas,United States,False,False,False
1,3167848617667650918,2,WC North Parking to Big Bridge,608,556,2023-12-07T13:56:58Z,2023-12-07T07:56:58Z,1311.2,16,145,...,"[30.401226, -97.684565]","[30.402613, -97.694794]",None,0,Austin,Texas,United States,False,False,False
2,3167848617667602790,2,Northern Walnut Creek Tr. 13,84,54,2023-12-07T14:00:19Z,2023-12-07T08:00:19Z,131.1,57,71,...,"[30.400007, -97.688264]","[30.399863, -97.68948]",None,0,Austin,Texas,United States,False,False,False
3,3167848617670688102,2,Logger Push,114,114,2023-12-07T14:02:53Z,2023-12-07T08:02:53Z,278.9,87,109,...,"[30.40124, -97.690204]","[30.403308, -97.691848]",None,0,Austin,Texas,United States,False,False,False
4,3167848617669508454,2,Northen Walnut Creek Tr. 12,114,114,2023-12-07T14:05:03Z,2023-12-07T08:05:03Z,258.1,113,144,...,"[30.403087, -97.692513]","[30.402439, -97.694707]",None,0,Austin,Texas,United States,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,3137525657584546178,2,Park Bridge to Lamar Bridge,92,92,2023-09-14T21:41:57Z,2023-09-14T16:41:57Z,270.2,67,88,...,"[30.395614, -97.682573]","[30.39486, -97.680118]",None,0,Austin,Texas,United States,False,False,False
168,3137525657588128130,2,Wind East from Lamar,88,88,2023-09-14T21:43:15Z,2023-09-14T16:43:15Z,268.7,84,110,...,"[30.394983, -97.680487]","[30.39388, -97.678701]",None,0,Austin,Texas,United States,False,False,False
169,3137525657588282754,2,WC Parking to 1st Bridge,82,82,2023-09-14T21:57:40Z,2023-09-14T16:57:40Z,253.8,292,311,...,"[30.401144, -97.684408]","[30.401298, -97.686722]",None,0,Austin,Texas,United States,False,False,False
170,3137525657587720578,2,Northern Walnut Creek Tr. 13,48,48,2023-09-14T22:00:16Z,2023-09-14T17:00:16Z,131.1,326,336,...,"[30.400007, -97.688264]","[30.399863, -97.68948]",None,0,Austin,Texas,United States,False,False,False
